## 4.1 Import packages


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

## 4.2 Input data
MNIST dataset [Click](http://yann.lecun.com/exdb/mnist/)

In [2]:
mnist = input_data.read_data_sets("data\\MNIST_data\\", one_hot=True)

W1014 10:18:18.224146 26500 deprecation.py:323] From <ipython-input-2-b64c65daac89>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1014 10:18:18.224146 26500 deprecation.py:323] From D:\APPs\Anaconda\envs\Jeary_lab\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1014 10:18:18.224146 26500 deprecation.py:323] From D:\APPs\Anaconda\envs\Jeary_lab\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for up

Extracting data\MNIST_data\train-images-idx3-ubyte.gz


W1014 10:18:18.536801 26500 deprecation.py:323] From D:\APPs\Anaconda\envs\Jeary_lab\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1014 10:18:18.542780 26500 deprecation.py:323] From D:\APPs\Anaconda\envs\Jeary_lab\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1014 10:18:18.609742 26500 deprecation.py:323] From D:\APPs\Anaconda\envs\Jeary_lab\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be rem

Extracting data\MNIST_data\train-labels-idx1-ubyte.gz
Extracting data\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting data\MNIST_data\t10k-labels-idx1-ubyte.gz


## 4.3 Set parameters

In [3]:
input_num = 784
h1_num = 300
epochs = 3000
batch_num = 100
lr = 0.1
kp = 0.75
W1 = tf.Variable(tf.truncated_normal([input_num, h1_num],stddev=0.1))
W2 = tf.Variable(tf.zeros([h1_num,10]))
B1 = tf.Variable(tf.zeros([h1_num]))
B2 = tf.Variable(tf.zeros([10]))
x = tf.placeholder(tf.float32, [None, input_num])
keep_prob = tf.placeholder(tf.float32)
label = tf.placeholder(tf.float32, [None,10])

## 4.4 Create model

In [4]:
# Feedforward
hidden1 = tf.nn.relu(tf.matmul(x,W1) + B1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + B2)

# Backforward
cross_entropy = tf.reduce_mean(-tf.reduce_sum(label * tf.log(y), reduction_indices=[1]))
train_prediction = tf.equal(tf.argmax(y,1), tf.argmax(label,1))
train_accuracy = tf.reduce_mean(tf.cast(train_prediction, tf.float32))
train_step = tf.train.AdagradOptimizer(lr).minimize(cross_entropy)

W1014 10:18:18.948899 26500 deprecation.py:506] From <ipython-input-4-58dc2ae46243>:3: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1014 10:18:19.073020 26500 deprecation.py:506] From D:\APPs\Anaconda\envs\Jeary_lab\lib\site-packages\tensorflow\python\training\adagrad.py:76: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## 4.5 Tensorboard

In [5]:
tf.summary.scalar('loss', cross_entropy)
tf.summary.scalar('accurcy', train_accuracy)
image = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image('image', image, 6)

<tf.Tensor 'image:0' shape=() dtype=string>

## 4.6 Delete exsited record

In [6]:
log_dir = "mnist_log"
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)

## 4.7 Create an session and train data

In [7]:
with tf.Session() as sess:
    merged = tf.summary.merge_all()
    start = time.time()
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter(log_dir, sess.graph)
    for epoch in range(epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_num)
        train_summary, _ = sess.run([merged, train_step], feed_dict={x: batch_xs, label: batch_ys, keep_prob:kp}) 
        writer.add_summary(train_summary, epoch)
    stop = time.time()
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(label,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    predict_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, label: mnist.test.labels, keep_prob:1.0})
    print("The accuracy of the test data：{:.2f}%".format(predict_acc * 100))
    print("The total time：{:.2f}s".format(stop-start))
writer.close()

The accuracy of the test data：97.51%
The total time：13.72s
